# Interpolación y ajuste de curvas (fiteo)  <a class="tocSkip">

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('presentation')
fsize= (9,6)

## Interpolación

Muchas veces tenemos mediciones de datos variando algún parámetro en las condiciones, y estos datos están medidos a intervalos mayores de los que deseamos. En estos casos es común tratar de inferir los valores que tendrían las mediciones para valores intermedios de nuestro parámetro. Una opción es interpolar los datos. Algunas facilidades para ello están en el subpaquete **interpolate** del paquete **Scipy**.

Generemos algunos "datos experimentales"

In [ ]:
def fmodel(x):
  return (np.sin(x))**2*np.exp(-(x/3.5)**2)

In [ ]:
x0 = np.linspace(0., 2*np.pi, 60)
y0 = fmodel(x0)
x = np.linspace(0., 2*np.pi, 8)
y = fmodel(x)

In [ ]:
plt.plot(x0,y0,'--k', label='función', alpha=0.5)
plt.plot(x,y,'o', markersize=12, label='puntos')
plt.legend(loc='best')

Acá hemos simulado datos con una función oscilante con un decaimiento exponencial.

Ahora, importamos el submódulo `interpolate` del módulo `scipy`, que nos permite interpolar los datos:

In [ ]:
from scipy import interpolate

La interpolación funciona en dos pasos. En el primer paso realizamos todos los cálculos y obtenemos la función interpolante, y en una segunda etapa utilizamos esa función para interpolar los valores en los nuevos puntos sobre el eje x que necesitamos.

Utilizamos los *arrays* `x` e `y` como los pocos "datos experimentales" obtenidos

In [ ]:
print(f"{x = }")

y creamos la función interpolante basada en estos puntos:

In [ ]:
interpol_lineal = interpolate.interp1d(x, y)

In [ ]:
interpol_lineal # función

Ahora, creamos un conjunto de puntos `x1` donde queremos evaluar la función interpolando entre datos medidos

In [ ]:
x1 = np.linspace(0, 2*np.pi, 33)
y1_l = interpol_lineal(x1)

In [ ]:
plt.plot(x0,y0, '-k', label='función', alpha=0.5)
plt.plot(x, y,'o',  markersize=12, label='datos')
plt.plot(x1, y1_l,'--s', markersize=7, label='interp-1d')
plt.legend(loc='best')

Como vemos, la función que creamos consiste de tramos rectos entre los puntos "datos". 
Para realizar interpolaciones lineales (una recta entre pares de puntos) también se puede utilizar la rutina `interp()` del módulo **Numpy**, cuyos argumentos requeridos son: los nuevos puntos `x1` donde queremos interpolar, además de los valores originales de `x` y de `y` de la tabla a interpolar:

In [ ]:
y1_l2= np.interp(x1,x,y)

Notar que `y1_l2` da exactamente los mismos valores que `y1_l`

In [ ]:
np.all(y1_l2 == y1_l)

Si bien el uso de `np.interp` es más directo, porque no se crea la función intermedia, cuando creamos la función con `interp1d` podemos aplicarla a diferentes conjuntos de valores de x:

In [ ]:
interpol_lineal(np.linspace(0, 2*np.pi, 12))

La interface `interp1d()` tiene un argumento opcional, `kind`, que define el tipo de interpolación a utilizar. Cuando utilizamos el argumento 'nearest' utiliza para cada valor el más cercano

In [ ]:
interpol_near = interpolate.interp1d(x, y, kind='nearest')
y1_n = interpol_near(x1)

In [ ]:
plt.plot(x0,y0, '-k', label='función', alpha=0.5)
plt.plot(x, y,'o', markersize=12, label='datos')
plt.plot(x1, y1_l,'--s', markersize=7, label='interp-1d')
plt.plot(x1, y1_n,'.', label='nearest')
plt.legend(loc='best');
print(x1.size, x1.size, x.size)

### Interpolación con polinomios

**Scipy** tiene rutinas para interpolar los datos usando un único polinomio global con grado igual al número de puntos dados:

In [ ]:
def fm(x):
  return x**4 - x**3*np.sin(x/6)

x0 = np.linspace(0., 2*np.pi, 60)
y0 = fm(x0)
x = np.linspace(0., 2*np.pi, 5)
y = fm(x)
# 
# Creamos el polinomio interpolador
f = interpolate.lagrange(x, y)
y1 = f(x0)
# 
plt.figure(figsize=fsize)
plt.plot(x0,y1,'-', label=f'Lagrange ({x.size})')
plt.plot(x,y,'o', label='datos')
plt.plot(x0,y0,'--', color='C1', label='f(x)', alpha=0.7)
plt.legend(loc='best')

Acá la función `lagrange()` devuelve un polinomio del tipo `poly1d`, que puede ser ejecutado (como hicimos en la celda anterior)

In [ ]:
type(f)

In [ ]:
f.coeffs

Los polinomios interpolantes pueden tener problemas, principalmente en las puntas, o cuando el grado del polinomio es muy alto. Consideremos por ejemplo el caso donde tenemos una tabla `x1  f(x1)` con muchos datos y queremos interpolar sobre una nueva tabla de valores `x0`. Incluso para un grado intermedio de polinomio se observan oscilaciones entre los puntos

In [ ]:
x0 = np.linspace(0., 2*np.pi, 60)
x1 = np.linspace(0, 2*np.pi, 7)
y1 = fmodel(x1)
f1 = interpolate.lagrange(x1, y1)
plt.figure(figsize=fsize)
plt.plot(x0,f1(x0),'-', label=f'Lagrange ({x1.size})')
plt.plot(x1,y1,'o', label='datos')
plt.plot(x0,fmodel(x0),'--', color='C1', label='f(x)', alpha=0.7)
plt.legend(loc='best')


In [ ]:
x0 = np.linspace(0., 4*np.pi, 60)
x1 = np.linspace(0, 4*np.pi, 21)
y1 = fmodel(x1)
f1 = interpolate.lagrange(x1, y1)
plt.figure(figsize=fsize)
plt.plot(x0,f1(x0),'-', label=f'Lagrange ({x1.size})')
plt.plot(x1,y1,'o', label='datos')
plt.plot(x0, interpolate.barycentric_interpolate(x1,y1,x0), label="Lagrange (estable)")
plt.plot(x0,fmodel(x0),'--', color='C1', label='f(x)', alpha=0.7)
plt.legend(loc='best')


De todas maneras, para los casos en que es aplicable, existen dos implementaciones: `interpolate.lagrange()` y una segunda llamada `interpolate.barycentric_interpolate()` que está basada en un trabajo de 2004 y es numéricamente más estable.

### Splines

-  Las *Splines* son interpolaciones por polinomios de a trazos, que se eligen para que no sólo los valores sino también sus derivadas coincidan dando una curva suave.
-  Para eso, si se pide que la aproximación coincida con los valores tabulados en los puntos dados, la aproximación es efectivamente una **interpolación**.
-  *Cubic Splines* se refiere a que utilizamos polinomios cúbicos en cada trozo.

El argumento opcional `kind` de la interface `interp1d()`, que define el tipo de interpolación a utilizar, acepta valores del tipo *string* que pueden ser: 'linear', 'nearest', 'zero', 'slinear', 'quadratic', 'cubic', o un número entero indicando el orden.

In [ ]:
x0 = np.linspace(0, 2*np.pi, 60)
x1 = np.linspace(0, 2*np.pi, 30)
interp = {}
for k in ['zero', 'slinear', 'quadratic', 'cubic']:
  interp[k] = interpolate.interp1d(x, y, kind=k)

In [ ]:
fig = plt.figure(figsize=fsize)
plt.plot(x0,y0,'-k', alpha=0.7, label='f(x)')
plt.plot(x,y,'o', markersize=12, label='datos')
plt.plot(x1,interp['cubic'](x1),  '--', color='C2', label=u'cúbica')
plt.plot(x1,interp['slinear'](x1),'.-', lw=2, label='lineal')
plt.legend(loc='best')

Tratamos de incluir todo en un sólo gráfico (y rogamos que se entienda algo) 

In [ ]:
plt.figure(figsize=fsize)
for k, v in interp.items():
  plt.plot(x1, v(x1), label=k)
plt.plot(x0,y0,'--k', alpha=0.7, label='f(x)')
plt.plot(x,y,'ob', markersize=12, label='datos')
plt.legend(loc='best')

En resumen, los métodos disponibles en `interpolate.interp1d` son:

- `linear`: Interpolación lineal, utilizando rectas (default)
- `nearest`         : Valor constante correspondiente al dato más cercano 
- `zero` o  `0`     : Una spline de orden cero. Toma el valor a la izquierda
- `slinear` o  `1`  : Spline de orden 1. Igual a 'linear' 
- `quadratic` o `2` : Spline de segundo orden 
- `cubic` o  `3`    : Spline de tercer orden 



Como vemos de los argumentos `zero`, `slinear`, `quadratic`, `cubic` para especificar splines de cero, primer, segundo y tercer orden se puede pasar como argumento un número. En ese caso se utiliza siempre **splines** y el número indica el orden de las splines a utilizar:

In [ ]:
for k,s in zip([0,1,2,3], ['zero','slinear','quadratic','cubic']):
  num = interpolate.interp1d(x,y, kind=k)
  tipo = interpolate.interp1d(x,y, kind=s)
  print(f"¿{k} == {s}? -> {np.allclose(num(x1), tipo(x1))}")

Además La interpolación lineal simple es, en la práctica, igual a la interpolación por splines de primer orden:

In [ ]:
interpol_lineal = interpolate.interp1d(x, y)
np.allclose(interp['slinear'](x1), interpol_lineal(x1)) # También son iguales

Finalmente, veamos que la interpolación `nearest` toma para cada nuevo valor `x1` el valor `y1(x1)` igual a `y(x0)` donde `x0` es el valor **más cercano** a `x1` mientras que la spline de orden cero (`zero`) toma el valor **más cercano a la izquierda del dato**:

In [ ]:
interpol_near = interpolate.interp1d(x, y, kind='nearest')
alfa=1
plt.figure(figsize=fsize)
plt.plot(x1, interpol_near(x1),'-o', label='nearest', alpha=alfa)
plt.plot(x1, interp['zero'](x1), '-.', label='Spline zero'.format(k), alpha=alfa)
plt.plot(x,y,'xk', markersize=12, label='datos')

plt.legend(loc='best');

El submódulo `signal` tiene rutinas adicionales para realizar *splines*, que permiten agregar un "alisado", pero en este caso ya no interpolan estrictamente sino que puede ser que la aproximación no pase por los puntos dados.

### B-Splines

Hay otra opción para realizar interpolación con Splines en Scipy. Las llamadas **B-Splines** son funciones diseñadas para generalizar polinomios, con un alto grado de **localidad**.

Para definir las **B-Splines** necesitamos dos cosas: 

  1. Elegir el grado de los polinomios (mayor o igual a 0)
  2. Dividir el intervalo en $n$ "nodos" 

Las funciones se definen mediante la recursión:

$$
N_{i, 0}(x) = 1,\qquad \qquad \textrm{si $t_i \le x < t_{i+1}, \qquad \qquad$ sino $0$,}
$$
$$
N_{i, k}(x) = \frac{x - t_i}{t_{i+k} - t_i} N_{i, k-1}(x)
         + \frac{t_{i+k+1} - x}{t_{i+k+1} - t_{i+1}} N_{i+1, k-1}(x)
$$

Las más simples, cuando el orden es k=0, son funciones constantes a trozos

![](figuras/bsplines0.png)

Para $k>0$ las funciones se calculan por recurrencia en término de dos funciones del orden anterior. Entonces, siempre serán diferentes de cero sólo en un intervalo finito. 
En ese intervalo presentan un único máximo y luego decaen suavemente. Las más usuales son las de orden $k=3$:

![](figuras/bsplines.png)

(Figura de http://www.brnt.eu/phd)



La idea es encontrar una función $f(x)$ que sea suave y pase por la tabla de puntos $(x,y)$ dados, o cerca de ellos con la condición que tanto la función como algunas de sus derivadas sea suave.
La función $f(x)$ se describe como una expansión en la base de Splines (y de ahí el nombre *B-Splines*) 

$$
f(x) = \sum_{j=0} a_{i,j} N_{j}(x) \qquad \forall \quad x_{i} < x \le x_{i+1}
$$

La aproximación se elige de tal manera de optimizar el número de elementos de la base a utilizar con la condición que el error cuadrático a los puntos sea menor a un cierto valor umbral $s$

$$
\sum_{i=1}^{n} \left| f(x_{i}) - y_{i}\right|^2 \le s
$$

Veamos cómo usar la implementación de **Scipy** para interpolar datos. En primer lugar creamos la representación en B-Splines de nuestra tabla de datos $(x,y)$:

In [ ]:
tck0 = interpolate.splrep(x,y)

Acá, otra vez estamos operando en dos pasos. En el primero creamos la representación de las splines para los datos dados.
Como no pusimos explícitamente el orden, utiliza el valor por default `k=3`.

En el segundo paso obtenemos los valores interpolados sobre la grilla `x2`:

In [ ]:
tck0

In [ ]:
x2 = np.linspace(0, 2*np.pi, 60) # Nuevos puntos donde interpolar
y_s0 = interpolate.splev(x2, tck0) # Valores interpolados: y_s0[j] = f(x2[j])

In [ ]:
plt.figure(figsize=fsize)
plt.plot(x,y,'o', markersize=12, label='datos')
plt.plot(x2,y_s0,'-', label=r'B-Spline')
plt.plot(x0,y0,'--k', alpha=0.7, label='f(x)')
plt.legend(loc='best');

Estas funciones interpolan los datos con curvas continuas y con derivadas segundas continuas.

### Lines are guides to the eyes

Sin embargo, estas rutinas no necesariamente realizan *interpolación* en forma estricta, pasando por todos los puntos dados, sino que en realidad realiza una aproximación minimzando por cuadrados mínimos la distancia a la tabla de puntos dados.

Esto es particularmente importante cuando tenemos datos que tienen dispersión. En esos casos necesitamos curvas que no interpolen, es decir que *no necesariamente* pasen por todos los puntos.

La rutina `splrep` tiene varios argumentos opcionales. Entre ellos un parámetro de suavizado `s` que corresponde a la condición de distancia entre la aproximación y los valores de la tabla mencionado anteriormente.  
Para ver como funciona, creemos una tabla de valores `x, y` 
con $x \in [0,2\pi]$ **no necesariamente equiespaciados**, $y=\sin(x)/2$, donde le agregamos algo de ruido a `y`

In [ ]:
# Creamos dos tablas de valores x, y
x3 = np.linspace(0., 2*np.pi, 40)
x4 = np.linspace(0., 2*np.pi, 40)
x3[5:-5] -= 0.7*(0.5-np.random.random(30)) # Le agregamos una separación al azar
x3.sort()                       # Los ordenamos 
plt.plot(x3-x4,'o',label='data')
plt.plot(x3-x3, '-', alpha=0.7,label='sin incerteza')
plt.ylim((-0.3,0.3));

In [ ]:
y4 = 0.5* np.sin(x4)
y3 = 0.5* np.sin(x3) * (1+ 0.6*(0.5-np.random.random(x3.size)))

In [ ]:
# Los puntos a interpolar tienen "ruido" en las dos direcciones x-y
plt.plot(x3,y3,'o', x4,y4, '.', alpha=0.7 );

In [ ]:
# Grilla donde evaluar la función interpolada
x1 = np.linspace(0, 2*np.pi, 90)

In [ ]:
tck0 = interpolate.splrep(x3,y3, s=0)  # Interpolación con B-Splines
y_s0 = interpolate.splev(x1,tck0)      # Evaluamos en la nueva grilla
tck3 = interpolate.splrep(x3,y3,s=0.3) # Aproximación suavizada
y_s3 = interpolate.splev(x1,tck3)      # Evaluamos en la nueva grilla

In [ ]:
plt.figure(figsize=fsize)
plt.plot(x1,y_s0,'--', lw=3, label=u'interpolación' )
plt.plot(x1,y_s3, "-",  label=u'suavizado');
plt.plot(x3,y3,'o', label='datos' )
plt.legend(loc='best');


El valor del parámetro `s` determina cuanto se suaviza la curva. El valor por default `s=0` obliga al algoritmo a obtener una solución que no difiere en los valores de la tabla, un valor de `s` mayor que cero da cierta libertad para obtener la aproximación que se acerque a todos los valores manteniendo una curva relativamente suave. El suavizado máximo corresponde a `s=1`. Veamos cómo cambia la aproximación con el factor `s`:

In [ ]:
tck1 = interpolate.splrep(x3,y3, s=0.1)  # Interpolación con suavizado
y_s1 = interpolate.splev(x1,tck1)
plt.figure(figsize=fsize)
plt.plot(x1,y_s1, "-",  label='s=0.1');
plt.plot(x1,y_s3, "--",  label='s=0.3');
plt.plot(x3,y3,'o', markersize=8, label='datos' )
plt.legend(loc='best');

In [ ]:
fig, (ax0, ax1) = plt.subplots(figsize=(2.1*fsize[0], fsize[1]), ncols=2)
for s in [0, 0.01, 0.05, 0.1, 0.2]:
  tck1 = interpolate.splrep(x3,y3, s=s)  # Interpolación con suavizado
  ys = interpolate.splev(x1,tck1)
  ax0.plot(x1,ys, "-",  label=f'{s =}');
  ax1.plot(x1,ys, "-",  label=f'{s =}');
ax0.plot(x3,y3,'ok', label='datos' )
ax1.plot(x3,y3,'ok', label='datos' )
plt.xlim(3,6)
ax1.set_ylim(-0.9, -0.2)
ax1.legend(loc='best');

### Cantidades derivadas de *splines*

De la interpolación (suavizada) podemos calcular, por ejemplo,  la derivada.

In [ ]:
yderiv = interpolate.splev(x1,tck3,der=1) #  Derivada

Si tenemos sólo los datos podríamos tratar de calcular la derivada en forma numérica como el coseno
$$ y' = 0.5 \sqrt{1 - (2y)^2} $$

Comparemos los dos resultados:

In [ ]:
cond = (x3 > np.pi/2) & (x3 < 3*np.pi/2)
yprima1 = np.where(cond, -1, 1) * 0.5*np.sqrt(np.abs(1 - (2*y3)**2))

In [ ]:
plt.figure(figsize=fsize)
plt.plot(x3, yprima1,"sr", label=r"con $\sqrt{1-(2y)^2}/2$")
plt.plot(x1,yderiv,'-k', label=u'Splines')
plt.legend(loc='best');

o  podemos calcular la integral, o las raíces de la función

In [ ]:
plt.figure(figsize=fsize)
yt= np.array([interpolate.splint(0,t,tck3) for t in x1])
plt.plot(x1,yt,'o');

In [ ]:
raices = interpolate.sproot(tck3)

In [ ]:
plt.axhline(0, color='k',ls='--');
plt.plot(x3,y3, "--",  label=u'datos');
plt.plot(x1,y_s3, "-",  label=u's=0.3');
plt.plot(raices, np.zeros(len(raices)), 'o', markersize=12)
plt.legend();

In [ ]:
# Generamos los datos

def f(x, y):
  s = np.hypot(x, y)          # Calcula la hipotenusa
  phi = np.arctan2(y, x)      # Calcula el ángulo
  tau = s + s*(1-s)/5 * np.sin(6*phi) 
  return 5*(1-tau) + tau

# Generamos los puntos x,y,z en una grilla para comparar con la interpolación
# Notar que es una grilla de 100x100 = 10000 puntos
#
x = np.linspace(-1,1,100)
y =  np.linspace(-1,1,100)
X, Y = np.meshgrid(x,y)
T = f(X, Y)


Aquí `T` contiene la función sobre todos los puntos $(x,y)$ obtenidos de combinar cada punto en el vector `x` con cada punto en el vector `y`. Notar que la cantidad total de puntos de `T` es `T.size = 10000`

Elegimos `npts=400` puntos al azar de la función que vamos a usar como tabla de datos a interpolar usando distintos métodos de interpolación

In [ ]:
npts = 400
px, py = np.random.choice(x, npts), np.random.choice(y, npts)
Z = f(px, py)

Para poder mostrar todos los métodos juntos, vamos a interpolar dentro del loop `for`

In [ ]:
# Graficación:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=fsize, )

# Graficamos la función sobre la grilla estructurada a modo de ilustración
# Graficamos los puntos seleccionados
ax[0,0].contourf(X, Y, T)
ax[0,0].scatter(px, py, c='k', alpha=0.6, marker='.', s=8)
ax[0,0].set_title('Puntos de f(X,Y)', fontsize='large')

# Interpolamos usando los distintos métodos y graficamos
for i, method in enumerate(('nearest', 'linear', 'cubic')):
  Ti = interpolate.griddata((px, py), Z, (X, Y), method=method)
  r, c = (i+1) // 2, (i+1) % 2
  ax[r,c].contourf(X, Y, Ti)
  ax[r,c].set_title('{}'.format(method), fontsize='large')
    
plt.subplots_adjust(hspace=0.3, wspace=0.3)

En la primera figura (arriba a la izquierda) graficamos la función evaluada en el total de puntos (10000 puntos) junto con los puntos utilizados para el ajuste. Los otros tres gráficos corresponden a la función evaluada siguiendo el ajuste correspondiente en cada caso.

## Fiteos de datos

### Ajuste con polinomios

Habitualmente realizamos ajustes sobre datos que tienen incertezas o ruido. Generemos estos datos (con ruido normalmente distribuido)

In [ ]:
plt.figure(figsize=fsize)
x = np.linspace(0, 10, 50)
y0 = 2.5*x + 1.2
ruido = np.random.normal(loc= 0., scale= 1, size= y0.size)
y = y0 + ruido
plt.plot(x,y0,'--')
plt.plot(x,y, 'ok', alpha=0.6)
plt.xlabel("Eje x")
plt.ylabel("Eje y");

Ahora vamos a ajustar con una recta

$$y = m x + b    \qquad \equiv \qquad    f(x) = p[0] x + p[1]$$

Es una regresión lineal (o una aproximación con polinomios de primer orden)


In [ ]:
p = np.polyfit(x,y,1)
# np.info(np.polyfit) # para obtener más información

In [ ]:
print(p)
print(type(p))                  # Qué tipo es?


---

**NOTA:**
¿Qué devuelve `polyfit()`?

Un array correspondiente a los coeficientes del polinomio de fiteo. En este caso, como estamos haciendo un ajuste lineal, nos devuelve los coeficientes de un polinomio de primer orden (una recta)
```python
y = p[0]*x + p[1]
```

---


In [ ]:
np.polyfit?

In [ ]:
plt.figure(figsize=fsize)
plt.plot(x, p[0]*x + p[1], '-', label='Ajuste')
plt.plot(x,y,'ok', label='datos', alpha=0.6)
plt.legend(loc='best');

Ahora en vez de escribir la recta explícitamente le pedimos a **numpy** que lo haga usando los coeficientes que encontramos mediante el fiteo (utilizando la función *polyval*)

```python
y = np.polyval(p,x)
```

In [ ]:
plt.figure(figsize=fsize)
plt.plot(x, np.polyval(p,x), '-', label='Ajuste')
plt.plot(x,y,'ok', label='datos', alpha=0.6)
plt.legend(loc='best');

Como vemos, arroja exactamente el mismo resultado.

Si los datos tienen mucho ruido lo que obtenemos es, por supuesto, una recta que pasa por la nube de puntos:

In [ ]:
y= y0 + 5*ruido
p = np.polyfit(x, y , 1)
plt.figure(figsize=fsize)
plt.plot(x,np.polyval(p,x),'--', label='Ajuste') 
plt.plot(x,y, 'ok', alpha=0.6, label='datos')
plt.legend(loc='best');

In [ ]:
p

Similarmente podemos usar polinomios de orden superior. Por ejemplo, para utilizar parábolas sólo tenemos que cambiar el orden `n` del polinomio en el argumento de `polyfit(x, y, n)`:

In [ ]:
# Generamos los datos
a = [2.5, 1.4, 3.]
y0 = np.polyval(a,x)
y = y0 + 10*ruido

In [ ]:
# Ajustamos con un polinomio de segundo grado
p = np.polyfit(x, y , 2)

In [ ]:
plt.figure(figsize=fsize)
plt.plot(x,y0,'-', label="$f(x)={0:.2}x^2 + {1:.2} x + {2:.2}$".format(*a)) 
plt.plot(x,np.polyval(p,x),'--', label="$P(x)={0:.2}x^2 + {1:.2} x + {2:.2}$".format(*p))
plt.plot(x,y,'ok', alpha=0.7,label='Datos')
plt.legend(loc='best');

-----

## Ejercicios 13 (b)

3. En el archivo co_nrg.dat se encuentran los datos de la posición de los máximos de un espectro de CO2 como función del número cuántico rotacional J (entero). Haga un programa que lea los datos. Los ajuste con polinomios (elija el orden adecuado) y grafique luego los datos (con símbolos) y el ajuste con una línea sólida roja. Además, debe mostrar los parámetros obtenidos para el polinomio.

-----


## Fiteos con funciones arbitrarias

Vamos ahora a fitear una función que no responde a la forma polinomial.  
El submódulo `optimize` del paquete `scipy` tiene rutinas para realizar ajustes de datos utilizando funciones arbitrarias

Utilicemos una función "complicada":

In [ ]:
# string definido para la leyenda
sfuncion= r'${0:.3}\, \sin ({1:.2}\, x {3:+.2f}) \, \exp(-{2:.2f} x)$'

def fit_func(x, a, b, c, d):
  y = a*np.sin(b*x-d)*np.exp(-c*x)
  return y

Generamos ahora "datos" con dispersión basados en la función

$$
f(x) = a \, \sin(b x - d) e^{-c x}
$$

In [ ]:
sfuncion

In [ ]:
sfuncion.format(1.,2.,3.,4.)

In [ ]:
x = np.linspace(0., 2*np.pi, 50)
y0 = fit_func(x, 1., 3., 1/3, 0 )
y = y0 + 0.1*ruido

y los graficamos

In [ ]:
plt.figure(figsize=fsize)
plt.plot(x,y0,'-',label="$f(x)$") 
plt.plot(x,y,'ok',alpha=0.5,label='Datos') # repeated from above
plt.xlabel("Eje x") # labels again
plt.ylabel("Eje y")
plt.legend(loc='best');

Ahora vamos a interpolar los datos utilizando funciones del paquete **Scipy**.
En primer lugar vamos a utilizar la función `curve_fit`:

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
# ¿Qué hace esta función?
help(curve_fit)

En su forma más simple toma los siguientes argumentos:


    Parameters
    ----------
    f : callable
        The model function, f(x, ...).  It must take the independent
        variable as the first argument and the parameters to fit as
        separate remaining arguments.
    xdata : An M-length sequence or an (k,M)-shaped array for functions with k predictors
        The independent variable where the data is measured.
    ydata : M-length sequence
        The dependent data --- nominally f(xdata, ...)
    p0 : None, scalar, or N-length sequence, optional
        Initial guess for the parameters.  If None, then the initial
        values will all be 1 (if the number of parameters for the function
        can be determined using introspection, otherwise a ValueError
        is raised).


El primero: `f` es la función que utilizamos para modelar los datos, y que dependerá de la variable independiente `x` y de los parámetros a ajustar.  
También debemos darle los valores tabulados en las direcciones $x$ (la variable independiente) e $y$ (la variable dependiente).  
Además, debido a las características del cálculo numérico que realiza suele ser muy importante darle valores iniciales a los parámetros que queremos ajustar.

Veamos lo que devuelve:


    Returns
    -------
    popt : array
        Optimal values for the parameters so that the sum of the squared error
        of ``f(xdata, *popt) - ydata`` is minimized
    pcov : 2d array
        The estimated covariance of popt.  The diagonals provide the variance
        of the parameter estimate.
    

El primer *array* tiene los parámetros para "best-fit", y el segundo da la estimación del error: la matriz de covarianza


Ya tenemos los valores a ajustar guardados en arrays `x` e `y`

In [ ]:
# initial_guess= None 
initial_guess= [1., -1., 1., 0.2]
params, p_covarianza = curve_fit(fit_func, x, y, initial_guess)

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(x,y0,'-', alpha=0.7,label="$f(x)$") 
plt.plot(x,y,'ok', alpha=0.5, label='Datos') # repeated from above
label=sfuncion.format(*params)
plt.plot(x,fit_func(x, *params), '--', label=label)
plt.xlabel("Eje x") # labels again
plt.ylabel("Eje y")
plt.legend(loc='best');
ylim = plt.ylim()
plt.ylim((ylim[0],1.2*ylim[1]));

Veamos otro ejemplo similar, con muchos datos y dispersión

In [ ]:
from scipy.optimize import least_squares

In [ ]:
least_squares?

In [ ]:
# Puntos "experimentales" con dispersión
x = np.linspace(0., 2*np.pi, 5000)
y0= fit_func(x, 1., 3., 1/3, 0 )
y = y0 + 0.2* np.random.normal(loc= 0., scale= 1, size= y0.size);
# Fiteo

In [ ]:
initial_guess= [1., 3., 1., 0.2]
params, p_covarianza = curve_fit(fit_func, x, y, initial_guess)

In [ ]:
# Graficación
plt.figure(figsize=fsize)
plt.plot(x,y0,'-b',label="$f(x)$") 
plt.plot(x,y,'.k',label='datos') # repeated from above
label=sfuncion.format(*params)
plt.plot(x,fit_func(x, *params), '-r', label=label)
plt.xlabel("Eje x") # labels again
plt.ylabel("Eje y")
plt.legend(loc='best');

La función `curve_fit()` que realiza el ajuste devuelve dos valores: El primero es un *array* con los valores de los parámetros obtenidos, y el segundo es un *array* con los valores correspondientes a la matriz de covarianza, cuyos elementos de la diagonal corresponden a la varianza para cada parámetro

In [ ]:
params

In [ ]:
p_covarianza.shape

In [ ]:
np.diagonal(p_covarianza)

Así, por ejemplo el primer parámetro (correspondiente a la amplitud `a`) toma en estos casos el valor:

In [ ]:
for j,v in enumerate(['a','b', 'c', 'd' ]):
  print("{} = {:.5g} ± {:.3g}".format(v, params[j], np.sqrt(p_covarianza[j,j])))

### Ejemplo: Fiteo de picos

Vamos a suponer que los datos son obtenidos mediante la repetición de mediciones

In [ ]:
# Realizamos 1000 mediciones, eso nos da una distribución de valores
r = np.random.normal(loc=6, size=1000)

In [ ]:
# Veamos qué obtuvimos
plt.plot(r,'.')

In [ ]:
y, x = np.histogram(r, bins=30, range=(2,10), density=True)
x = (x[1:]+x[:-1])/2        # Calculamos los centros de los intervalos 

Vamos a graficar el histograma junto con la función Gaussiana con el valor correspondiente de la "Función Densidad de Probabilidad" (pdf)

In [ ]:
from scipy import stats
b = stats.norm.pdf(x, loc=6)
plt.figure(figsize=fsize)
plt.plot(x,b,'-', label=u'Función')
plt.plot(x,y,'o-', alpha=0.7, label='random')
plt.legend(loc='best');


Usando esta idea generemos "datos" correspondiente a dos picos, que son diferentes y están separados pero que no podemos distinguirlos completamente porque se superponen.

Generemos los datos:

In [ ]:
npoints = 3000
r = np.hstack([np.random.normal(size=npoints), np.random.normal(loc=2, scale=.6, size=npoints)])
y,x = np.histogram(r , bins = 40, range = (-3.5,4.5), density=True)
x = (x[1:]+x[:-1])/2

In [ ]:
r.shape, r.size

In [ ]:
plt.figure(figsize=fsize)
plt.plot(x,y,'o--k', alpha=0.6, label='"Medición"')
plt.legend(loc='best');


Ahora, por razones físicas (o porque no tenemos ninguna idea mejor) suponemos que esta curva corresponde a dos "picos" del tipo Gaussiano, sólo que no conocemos sus posiciones, ni sus anchos ni sus alturas relativas. Creamos entonces una función que describa esta situación: La suma de dos Gaussianas, cada una de ellas con su posición y ancho característico, y un factor de normalización diferente para cada una de ellas. En total tendremos seis parámetros a optimizar:

In [ ]:
def modelo(x, *coeffs):
  "Suma de dos Gaussianas, con pesos dados por coeffs[0] y coeffs[3], respectivamente"
  G = stats.norm.pdf
  return coeffs[0]*G(x,loc=coeffs[1], scale=coeffs[2]) + \
  coeffs[3]*G(x,loc=coeffs[4], scale=coeffs[5])

In [ ]:
help(modelo)

Ahora es muy fácil realizar el fiteo. Mirando el gráfico proponemos valores iniciales, donde los tres primeros valores corresponde a los parámetros de la primera Gaussiana y los tres últimos a los de la segunda:

In [ ]:
c0 = np.array([1., -0.5, 1., 1., 1.5, 1.])
c, cov = curve_fit(modelo, x, y, p0 = c0)
print(c)

In [ ]:
plt.figure(figsize=fsize)
plt.plot(x, y,'ok', alpha=0.8, label='random distrib')
plt.plot(x, modelo(x,*c0), '--', alpha=0.9, label='Curva inicial')
plt.plot(x, modelo(x,*c), '-', label='fiteo')
plt.plot(x,c[0]*stats.norm.pdf(x,loc=c[1], scale=c[2]), '--', alpha=0.7, label='Gaussiana 1')
plt.plot(x,c[3]*stats.norm.pdf(x,loc=c[4], scale=c[5]), '--', alpha=0.7, label='Gaussiana 2')
plt.legend( loc = 'best' );


-----

## Ejercicios 13 (c)

4. **Para Entregar:** Queremos hacer un programa que permita fitear una curva como suma de `N` funciones gaussianas:

    1. Usando distribuciones normales e histogramas cree conjuntos de datos con dos, tres y cuatro máximos.
    1. Haga una función, que debe tomar como argumento los arrays con los datos: `x`, `y`, y valores iniciales para las Gaussianas: `fit_gaussianas(x, y, *params)` donde params son los 3N coeficientes (tres coeficientes para cada Gaussiana). Debe devolver los parámetros óptimos obtenidos.
    2. Realice un programita que grafique los datos dados y la función que resulta de sumar las gaussianas en una misma figura.
    3. *Si puede* agregue líneas o flechas indicando la posición del máximo y el ancho de cada una de las Gaussianas obtenidas.

-----